# AutoShorts 모듈별 테스트 노트북

각 Agent를 개별적으로 테스트하고 프롬프트를 확인할 수 있습니다.

## 📋 필요한 API 키

`.env` 파일에 다음 키들을 설정하세요:

```bash
# 필수
OPENAI_API_KEY=sk-...           # GPT-4o-mini (플롯 생성)
REDIS_URL=redis://localhost:6379

# 권장
ELEVENLABS_API_KEY=...          # ElevenLabs (음성+음악 생성, $5/월~)

# 선택 (없으면 Stub 모드로 동작)
COMFYUI_URL=http://localhost:8188  # ComfyUI (이미지 생성, 로컬 무료)
MUBERT_LICENSE=...              # Mubert (음악 폴백, $14/월~, deprecated)
PLAYHT_USER_ID=...              # PlayHT (음성 대체, $31/월~)
PLAYHT_API_KEY=...
```

**비용 최적화 팁:**
- ElevenLabs 하나로 음성+음악 모두 생성 가능 ($5/월 플랜)
- Mubert는 비쌈 ($14/월), ElevenLabs로 대체 권장
- ComfyUI는 로컬 실행 시 무료 (GPU 필요)

## 🚀 실행 전 체크리스트

1. Redis 서버 실행: `redis-server`
2. ComfyUI 실행 (선택): `cd ComfyUI && python main.py`
3. 가상환경 활성화: `source kvenv/bin/activate`
4. 필요한 패키지 설치 확인: `pip install -r backend/requirements.txt`

## 0. 환경 설정

In [20]:
!pip install -r backend/requirements.txt


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [21]:
# Python path 설정 (backend 모듈 import 가능하도록)
import sys
from pathlib import Path

backend_path = Path.cwd() / "backend"
if str(backend_path) not in sys.path:
    sys.path.insert(0, str(backend_path))

print(f"✅ Python path 설정 완료: {backend_path}")

✅ Python path 설정 완료: /Users/yuli/Documents/01.AI_Workspace/Kurz_Studio_AI/backend


In [22]:
# 필요한 모듈 import
import json
from datetime import datetime
from pathlib import Path

print("✅ 기본 모듈 import 완료")

✅ 기본 모듈 import 완료


In [23]:
# API 키 확인
from app.config import settings

print("=" * 50)
print("API 키 상태 확인")
print("=" * 50)

api_status = {
    "OpenAI (플롯 생성)": "✅" if settings.OPENAI_API_KEY else "❌",
    "ComfyUI (이미지)": "✅" if settings.COMFY_URL else "⚠️  Stub 모드",
    "ElevenLabs (음성+음악)": "✅" if settings.ELEVENLABS_API_KEY else "⚠️  Stub 모드",
    "Mubert (음악 폴백)": "✅" if settings.MUBERT_LICENSE else "⚠️  Stub 모드",
    "PlayHT (음성 대체)": "✅" if settings.PLAYHT_API_KEY else "⚠️  Stub 모드",
    "Redis": "✅" if settings.REDIS_URL else "❌",
}

for service, status in api_status.items():
    print(f"{service:30s}: {status}")

print("\n💡 권장 설정:")
print("  - ELEVENLABS_API_KEY: 음성+음악 통합 (저렴, $5/월~)")
print("  - MUBERT_LICENSE: 음악 전용 폴백 (비쌈, $14/월~)")
print("\n⚠️  Stub 모드: API 키가 없으면 더미 데이터 생성")

API 키 상태 확인
OpenAI (플롯 생성)                : ✅
ComfyUI (이미지)                 : ✅
ElevenLabs (음성+음악)            : ✅
Mubert (음악 폴백)                : ⚠️  Stub 모드
PlayHT (음성 대체)                : ⚠️  Stub 모드
Redis                         : ✅

💡 권장 설정:
  - ELEVENLABS_API_KEY: 음성+음악 통합 (저렴, $5/월~)
  - MUBERT_LICENSE: 음악 전용 폴백 (비쌈, $14/월~)

⚠️  Stub 모드: API 키가 없으면 더미 데이터 생성


## 1. 플롯 생성 테스트 (스키마 v3: characters.json + plot.csv → layout.json)

GPT-4o-mini를 사용하여:
1. **캐릭터 정의 생성** (characters.json): 외형, 성격, seed
2. **플롯 생성** (plot.csv): 씬별 대사, 표정, 포즈
3. **JSON 변환** (layout.json): appearance + expression + pose 조합

### 스키마 v3 주요 변경사항
- ✅ **캐릭터 정의 분리**: characters.json (재사용 가능)
- ✅ **expression, pose 추가**: 씬별 세밀한 이미지 제어
- ❌ **char_name 제거**: characters.json에서 char_id로 참조
- 🎨 **이미지 생성**: appearance + expression + pose 자동 조합

In [24]:
# 테스트 파라미터 설정
test_prompt = "우주를 여행하는 용감한 고양이의 모험"
num_characters = 2
num_cuts = 5
mode = "story"

# Run ID 생성 (실제와 동일한 형식)
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
prompt_clean = "".join(c for c in test_prompt if not c.isspace())[:8]
run_id = f"{timestamp}_{prompt_clean}"

print(f"Run ID: {run_id}")
print(f"프롬프트: {test_prompt}")
print(f"캐릭터 수: {num_characters}")
print(f"장면 수: {num_cuts}")
print(f"모드: {mode}")

Run ID: 20251105_1557_우주를여행하는용
프롬프트: 우주를 여행하는 용감한 고양이의 모험
캐릭터 수: 2
장면 수: 5
모드: story


In [25]:
# 캐릭터 및 플롯 생성 (스키마 v3)
from app.utils.plot_generator import generate_plot_with_characters

print("=" * 50)
print("GPT-4o-mini에게 전달되는 프롬프트 (2단계)")
print("=" * 50)

# 1단계: 캐릭터 정의 생성
char_system_prompt = f"""당신은 숏폼 영상 콘텐츠의 캐릭터 디자이너입니다.
사용자의 요청에 맞는 {num_characters}명의 캐릭터를 만들어주세요.

각 캐릭터마다 다음 정보를 JSON 형식으로 제공하세요:
- char_id: char_1, char_2, ... 형식
- name: 캐릭터 이름 (창의적으로)
- appearance: 외형 묘사 (이미지 생성에 사용됩니다. 상세하게 작성)
- personality: 성격/특징
- voice_profile: "default" (고정값)
- seed: char_1은 1002, char_2는 1003, ... 순서대로

**중요**:
- 반드시 JSON 형식으로만 출력하세요
- appearance는 이미지 생성 프롬프트로 사용되므로 시각적 특징을 상세히 작성
"""

print("[1단계: 캐릭터 정의 생성]")
print(char_system_prompt[:500] + "...")

# 2단계: 플롯 생성
plot_system_prompt = f"""당신은 숏폼 영상 콘텐츠 시나리오 작가입니다.
사용자의 요청을 {num_cuts}개 장면으로 나누어 {'스토리' if mode == 'story' else '광고 콘텐츠'}를 만들어주세요.

각 장면마다 다음을 포함해야 합니다:
- scene_id: scene_1, scene_2, ... 형식
- char_id: char_1, char_2 (등장인물 ID)
- expression: 표정 (happy, sad, excited, neutral, surprised, angry 등)
- pose: 포즈 (standing, sitting, walking, pointing, jumping 등)
- text: 대사 또는 해설 내용
- text_type: dialogue (대사) 또는 narration (해설)
- emotion: neutral, happy, sad, excited, angry, surprised 중 하나
- subtitle_position: top 또는 bottom
- duration_ms: 장면 지속시간 (보통 4000-6000)

**중요**:
- 반드시 CSV 형식으로만 출력하세요
- expression과 pose는 이미지 생성에 사용됩니다
"""

print("\n[2단계: 플롯 생성]")
print(plot_system_prompt[:500] + "...")

print("\n" + "=" * 50)
print(f"사용자 프롬프트: {test_prompt}")
print("=" * 50)

# 캐릭터 및 플롯 생성 실행
print("\n🔄 캐릭터 및 플롯 생성 중...")
characters_path, csv_path = generate_plot_with_characters(
    run_id=run_id,
    prompt=test_prompt,
    num_characters=num_characters,
    num_cuts=num_cuts,
    mode=mode,
)

print(f"\n✅ 캐릭터 JSON 생성 완료: {characters_path}")
print(f"✅ 플롯 CSV 생성 완료: {csv_path}")

GPT-4o-mini에게 전달되는 프롬프트 (2단계)
[1단계: 캐릭터 정의 생성]
당신은 숏폼 영상 콘텐츠의 캐릭터 디자이너입니다.
사용자의 요청에 맞는 2명의 캐릭터를 만들어주세요.

각 캐릭터마다 다음 정보를 JSON 형식으로 제공하세요:
- char_id: char_1, char_2, ... 형식
- name: 캐릭터 이름 (창의적으로)
- appearance: 외형 묘사 (이미지 생성에 사용됩니다. 상세하게 작성)
- personality: 성격/특징
- voice_profile: "default" (고정값)
- seed: char_1은 1002, char_2는 1003, ... 순서대로

**중요**:
- 반드시 JSON 형식으로만 출력하세요
- appearance는 이미지 생성 프롬프트로 사용되므로 시각적 특징을 상세히 작성
...

[2단계: 플롯 생성]
당신은 숏폼 영상 콘텐츠 시나리오 작가입니다.
사용자의 요청을 5개 장면으로 나누어 스토리를 만들어주세요.

각 장면마다 다음을 포함해야 합니다:
- scene_id: scene_1, scene_2, ... 형식
- char_id: char_1, char_2 (등장인물 ID)
- expression: 표정 (happy, sad, excited, neutral, surprised, angry 등)
- pose: 포즈 (standing, sitting, walking, pointing, jumping 등)
- text: 대사 또는 해설 내용
- text_type: dialogue (대사) 또는 narration (해설)
- emotion: neutral, happy, sad, excited, angry, surprised 중 하나
- subtitle_position: top 또는 bottom
- duration_ms: 장면 지속시간 (보통 4000-6000)

**중요**:
- 반드시 ...

사용자 프롬프트: 우주를 여행하는 용감한 고양이의 모험

🔄 캐릭터 및 플롯 생성 중...

✅ 캐릭터 J

In [26]:
# 생성된 파일 확인 (스키마 v3)
import pandas as pd
import json

print("=" * 50)
print("1. 생성된 characters.json 내용")
print("=" * 50)

with open(characters_path, "r", encoding="utf-8") as f:
    characters_data = json.load(f)

for char in characters_data["characters"]:
    print(f"\n[{char['char_id']}] {char['name']}")
    print(f"  외형: {char['appearance']}")
    print(f"  성격: {char['personality']}")
    print(f"  음성: {char['voice_profile']}")
    print(f"  Seed: {char['seed']}")

print("\n" + "=" * 50)
print("2. 생성된 plot.csv 내용 (v3 스키마)")
print("=" * 50)

df = pd.read_csv(csv_path)
print(df.to_string())

print(f"\n총 {len(df)}개 행 생성됨")

print("\n" + "=" * 50)
print("스키마 v3 특징")
print("=" * 50)
print("✅ characters.json: 캐릭터 외형 및 설정 (appearance, personality, seed)")
print("✅ plot.csv: 장면별 표정/포즈 제어 (expression, pose)")
print("✅ char_name 제거: characters.json에서 char_id로 참조")
print("✅ 이미지 생성: appearance + expression + pose 조합")

1. 생성된 characters.json 내용

[char_1] 스타 캣
  외형: 푸른색과 보라색이 섞인 털을 가진 고양이로, 눈은 빛나는 별처럼 반짝인다. 우주복을 입고 있으며, 헬멧은 투명한 유리로 되어 있어 고양이의 귀여운 얼굴이 보인다. 등에는 작은 로켓 추진기가 장착되어 있어 공중에서도 자유롭게 날 수 있다. 앞발에는 우주 탐사 도구들이 부착되어 있다.
  성격: 호기심이 많고 용감하며, 새로운 모험을 두려워하지 않는다. 항상 긍정적인 태도를 유지하며, 친구들을 돕는 것을 좋아한다. 때때로 장난꾸러기 같은 면모를 보여주기도 한다.
  음성: default
  Seed: 1002

[char_2] 코스모 푸들
  외형: 우주를 배경으로 한 독특한 털색을 가진 푸들로, 은색과 우주 블루 색상이 섞여 있다. 눈은 크고 반짝이며, 귀는 별 모양으로 뾰족하다. 머리에는 우주 탐사 헬멧을 쓰고 있으며, 등에는 작은 우주선 모양의 배낭이 달려 있다. 발에는 작은 부스터가 부착되어 있어 빠르게 이동할 수 있다.
  성격: 지혜롭고 침착하며, 고양이 친구 스타 캣의 모험을 돕는 역할을 한다. 문제 해결 능력이 뛰어나며, 항상 상황을 분석하고 적절한 조언을 한다. 친구를 위해 위험을 감수할 줄 아는 충성심이 강하다.
  음성: default
  Seed: 1003

2. 생성된 plot.csv 내용 (v3 스키마)
  scene_id char_id expression        pose                              text  text_type    emotion subtitle_position  duration_ms
0  scene_1  char_1    excited    standing    고양이 스타 캣이 우주선을 타고 우주를 향해 출발해요!  narration      happy               top         5000
1  scene_2  char_2      happy     sitting       우와! 정말 멋

In [27]:
# CSV + characters.json → layout.json 변환 (스키마 v3)
from app.utils.json_converter import convert_plot_to_json

print("🔄 characters.json + plot.csv → layout.json 변환 중...")

json_path = convert_plot_to_json(
    csv_path=str(csv_path),
    run_id=run_id,
    art_style="파스텔 수채화",
    music_genre="ambient",
)

print(f"✅ JSON 생성 완료: {json_path}")

print("\n💡 변환 과정:")
print("  1. characters.json 읽기 → Character 객체")
print("  2. plot.csv 읽기 → Scene 객체")
print("  3. 이미지 프롬프트 생성: appearance + expression + pose")
print("  4. layout.json 저장")

🔄 characters.json + plot.csv → layout.json 변환 중...
✅ JSON 생성 완료: app/data/outputs/20251105_1557_우주를여행하는용/layout.json

💡 변환 과정:
  1. characters.json 읽기 → Character 객체
  2. plot.csv 읽기 → Scene 객체
  3. 이미지 프롬프트 생성: appearance + expression + pose
  4. layout.json 저장


In [28]:
# JSON 내용 확인 (스키마 v3)
with open(json_path, "r", encoding="utf-8") as f:
    layout = json.load(f)

print("=" * 50)
print("JSON 레이아웃 구조 (v3)")
print("=" * 50)

print(f"\nProject ID: {layout['project_id']}")
print(f"Title: {layout['title']}")
print(f"Total Duration: {layout['timeline']['total_duration_ms']}ms")

print(f"\n캐릭터 ({len(layout['characters'])}명):")
for char in layout["characters"]:
    print(f"  - {char['name']} (ID: {char['char_id']}, Seed: {char['seed']})")
    print(f"    Persona: {char['persona'][:50]}...")

print(f"\n씬 ({len(layout['scenes'])}개):")
for scene in layout["scenes"]:
    print(f"\n  [{scene['scene_id']}] ({scene['duration_ms']}ms)")
    print(f"    이미지 슬롯: {len(scene['images'])}개")
    
    # 이미지 프롬프트 확인 (appearance + expression + pose)
    if scene['images']:
        img = scene['images'][0]
        print(f"      이미지 URL: {img['image_url']}")
        print(f"      ref_id: {img.get('ref_id', 'N/A')}")
    
    print(f"    대사: {len(scene['dialogue'])}개")
    if scene['dialogue']:
        for dialogue in scene['dialogue']:
            print(f"      대사: {dialogue['text'][:40]}...")
    
    print(f"    자막: {len(scene['subtitles'])}개")
    # 자막 내용 확인 (text_type에 따른 큰따옴표 추가 확인)
    if scene["subtitles"]:
        for subtitle in scene["subtitles"]:
            print(f"      자막 텍스트: {subtitle['text'][:50]}...")

print("\n" + "=" * 50)
print("스키마 v3 주요 특징")
print("=" * 50)
print("✅ characters.json 분리: 캐릭터 외형 재사용")
print("✅ plot.csv expression/pose: 씬별 표정/포즈 제어")
print("✅ 이미지 프롬프트 자동 생성: appearance + expression + pose")
print("✅ text_type=dialogue → 자막에 큰따옴표 자동 추가")

JSON 레이아웃 구조 (v3)

Project ID: 20251105_1557_우주를여행하는용
Title: AutoShorts 20251105_1557_우주를여행하는용
Total Duration: 26000ms

캐릭터 (2명):
  - 스타 캣 (ID: char_1, Seed: 1002)
    Persona: 호기심이 많고 용감하며, 새로운 모험을 두려워하지 않는다. 항상 긍정적인 태도를 유지하며,...
  - 코스모 푸들 (ID: char_2, Seed: 1003)
    Persona: 지혜롭고 침착하며, 고양이 친구 스타 캣의 모험을 돕는 역할을 한다. 문제 해결 능력이 뛰...

씬 (5개):

  [scene_1] (5000ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_1
    대사: 1개
      대사: 고양이 스타 캣이 우주선을 타고 우주를 향해 출발해요!...
    자막: 1개
      자막 텍스트: 고양이 스타 캣이 우주선을 타고 우주를 향해 출발해요!...

  [scene_2] (4000ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_2
    대사: 1개
      대사: 우와! 정말 멋진 우주야! 별들이 반짝이고 있어!...
    자막: 1개
      자막 텍스트: "우와! 정말 멋진 우주야! 별들이 반짝이고 있어!"...

  [scene_3] (6000ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_1
    대사: 1개
      대사: 우리는 우주를 탐험할 준비가 되어 있어!...
    자막: 1개
      자막 텍스트: 우리는 우주를 탐험할 준비가 되어 있어!...

  [scene_4] (5000ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_2
    대사: 1개
      대사: 저기! 새로운 행성이 나타났어! 

## 2. 이미지 생성 테스트 (Banana / ComfyUI)

이미지 생성 프로바이더:
- **Banana** (권장): 서버리스 GPU, API 키만으로 즉시 사용 가능
- **ComfyUI**: 로컬 실행, 무료 (GPU 필요, `http://localhost:8188`)

In [ ]:
# 이미지 생성 클라이언트 초기화
from app.config import settings

print("=" * 50)
print("이미지 생성 프로바이더 확인")
print("=" * 50)

provider = settings.IMAGE_PROVIDER
print(f"\n설정된 프로바이더: {provider}")

if provider == "banana":
    # Banana 설정 확인
    has_api_key = bool(settings.BANANA_API_KEY)
    has_model_key = bool(settings.BANANA_MODEL_KEY)
    
    print(f"BANANA_API_KEY: {'✅ 설정됨' if has_api_key else '❌ 미설정'}")
    print(f"BANANA_MODEL_KEY: {'✅ 설정됨' if has_model_key else '❌ 미설정'}")
    
    if has_api_key and has_model_key:
        from app.providers.images.banana_client import BananaClient
        image_client = BananaClient(
            api_key=settings.BANANA_API_KEY,
            model_key=settings.BANANA_MODEL_KEY
        )
        print(f"\n✅ Banana 클라이언트 초기화 완료")
    else:
        print(f"\n⚠️  Banana API 키가 설정되지 않음. .env 파일에 추가하세요:")
        print(f"   BANANA_API_KEY=your_api_key")
        print(f"   BANANA_MODEL_KEY=your_model_key")
        image_client = None

elif provider == "comfyui":
    # ComfyUI 연결 확인
    import httpx
    try:
        response = httpx.get(f"{settings.COMFY_URL}/system_stats", timeout=2.0)
        if response.status_code == 200:
            from app.providers.images.comfyui_client import ComfyUIClient
            image_client = ComfyUIClient()
            print(f"\n✅ ComfyUI 클라이언트 초기화 완료: {settings.COMFY_URL}")
        else:
            raise Exception("ComfyUI not responding")
    except:
        print(f"\n⚠️  ComfyUI 서버 연결 실패: {settings.COMFY_URL}")
        print(f"   1. ComfyUI 실행 여부 확인")
        print(f"   2. 또는 Banana로 변경: .env에서 IMAGE_PROVIDER=banana")
        image_client = None
else:
    print(f"\n❌ 알 수 없는 프로바이더: {provider}")
    image_client = None

In [ ]:
# 첫 번째 씬의 이미지 생성 테스트 (스키마 v3)
scene = layout["scenes"][0]
char = layout["characters"][0]

print("=" * 50)
print("이미지 생성 테스트")
print("=" * 50)

# CSV에서 expression/pose 읽기
df = pd.read_csv(csv_path)
first_scene = df[df['scene_id'] == scene['scene_id']].iloc[0]

# 프롬프트 구성 (v3 방식)
art_style = layout["metadata"]["art_style"]
appearance = char["persona"]  # characters.json의 persona (실제로는 appearance 필드)
expression = first_scene['expression']
pose = first_scene['pose']
char_seed = char["seed"]
bg_seed = scene["bg_seed"]

# 이미지 프롬프트: appearance + expression + pose
if expression != "none" and pose != "none":
    image_prompt = f"{art_style}, {appearance}, {expression} expression, {pose} pose"
else:
    image_prompt = f"{art_style}, {appearance}"

print(f"캐릭터: {char['name']}")
print(f"Appearance: {appearance}")
print(f"Expression: {expression}")
print(f"Pose: {pose}")
print(f"Character Seed: {char_seed}")
print(f"Background Seed: {bg_seed}")
print(f"\n최종 프롬프트: {image_prompt}")

if image_client:
    print(f"\n🔄 이미지 생성 중 ({provider} 사용)...")
    
    try:
        if provider == "banana":
            image_path = image_client.generate_image(
                prompt=image_prompt,
                seed=char_seed,
                width=512,
                height=768,
                output_prefix=f"app/data/outputs/{run_id}/scene_1_center",
            )
        elif provider == "comfyui":
            image_path = image_client.generate_image(
                prompt=image_prompt,
                seed=char_seed,
                output_prefix=f"app/data/outputs/{run_id}/scene_1_center",
            )

        print(f"\n✅ 이미지 생성 완료: {image_path}")

        # 이미지 표시 (Jupyter에서)
        from IPython.display import Image, display

        if Path(image_path).exists():
            display(Image(filename=image_path, width=400))

    except Exception as e:
        print(f"\n❌ 이미지 생성 실패: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"\n⚠️  이미지 클라이언트가 초기화되지 않음")
    print(f"   .env 파일에서 IMAGE_PROVIDER와 API 키를 확인하세요")

print("\n💡 v3 스키마:")
print("  - appearance (characters.json): 기본 외형")
print("  - expression (plot.csv): 씬별 표정")
print("  - pose (plot.csv): 씬별 포즈")
print("  → 조합하여 세밀한 이미지 제어 가능")

## 3. 음악 생성 테스트 (ElevenLabs / Mubert)

ElevenLabs Sound Effects API를 우선 사용하고, 없으면 Mubert를 폴백으로 사용합니다.

In [16]:
# Music 클라이언트 선택 (ElevenLabs 우선, Mubert 폴백)
from app.config import settings

if settings.ELEVENLABS_API_KEY:
    from app.providers.music.elevenlabs_music_client import ElevenLabsMusicClient

    music_client = ElevenLabsMusicClient(api_key=settings.ELEVENLABS_API_KEY)
    print("✅ ElevenLabs Music 클라이언트 초기화 완료")
    print("💡 ElevenLabs Sound Effects API 사용 (TTS와 통합, 저렴)")
elif settings.MUBERT_LICENSE:
    from app.providers.music.mubert_client import MubertClient

    music_client = MubertClient(api_key=settings.MUBERT_LICENSE)
    print("✅ Mubert 클라이언트 초기화 완료 (폴백)")
else:
    from app.providers.music.stub_client import StubMusicClient

    music_client = StubMusicClient()
    print("⚠️  Stub 모드로 실행 (음악 API 없음)")

ModuleNotFoundError: No module named 'requests'

In [17]:
# 배경음악 생성
genre = layout["metadata"]["music_genre"]
duration_ms = layout["timeline"]["total_duration_ms"]
duration_sec = duration_ms / 1000

print("=" * 50)
print("음악 생성 파라미터")
print("=" * 50)
print(f"장르: {genre}")
print(f"길이: {duration_sec}초 ({duration_ms}ms)")
print(f"무드: cinematic")

# ElevenLabs 사용 시 프롬프트 표시
if hasattr(music_client, "_build_music_prompt"):
    # ElevenLabs 클라이언트인 경우
    prompt = music_client._build_music_prompt(genre, "cinematic", duration_sec)
    print(f"\n[ElevenLabs Sound Effects API 프롬프트]")
    print(f"{prompt}")
    print(f"\nAPI 파라미터:")
    print(f"  - duration_seconds: {min(duration_sec, 22)} (최대 22초)")
    print(f"  - prompt_influence: 0.3")
elif hasattr(music_client, "generate"):
    # Mubert 클라이언트인 경우
    print(f"\n[Mubert API 파라미터]")
    print(f"  - genre: {genre}")
    print(f"  - mood: cinematic, dreamy")

print("\n🔄 음악 생성 중...")

try:
    audio_path = music_client.generate_music(
        genre=genre,
        mood="cinematic",
        duration_ms=duration_ms,
        output_filename=f"app/data/outputs/{run_id}/global_bgm.mp3",
    )

    print(f"\n✅ 음악 생성 완료: {audio_path}")

    # 오디오 재생 (Jupyter에서)
    from IPython.display import Audio, display

    if Path(audio_path).exists():
        display(Audio(filename=audio_path))

except Exception as e:
    print(f"\n❌ 음악 생성 실패: {e}")
    import traceback

    traceback.print_exc()

음악 생성 파라미터
장르: ambient
길이: 27.0초 (27000ms)
무드: cinematic


NameError: name 'music_client' is not defined

## 4. 음성 생성 테스트 (ElevenLabs / PlayHT)

TTS API를 사용하여 캐릭터 대사를 생성합니다.

In [ ]:
# TTS 클라이언트 선택
from app.config import settings

if settings.ELEVENLABS_API_KEY:
    from app.providers.tts.elevenlabs_client import ElevenLabsClient

    tts_client = ElevenLabsClient(api_key=settings.ELEVENLABS_API_KEY)
    print("✅ ElevenLabs 클라이언트 초기화 완료")
elif settings.PLAYHT_API_KEY:
    from app.providers.tts.playht_client import PlayHTClient

    tts_client = PlayHTClient(
        user_id=settings.PLAYHT_USER_ID,
        api_key=settings.PLAYHT_API_KEY
    )
    print("✅ PlayHT 클라이언트 초기화 완료")
else:
    from app.providers.tts.stub_client import StubTTSClient

    tts_client = StubTTSClient()
    print("⚠️  Stub 모드로 실행 (TTS 없음)")

In [19]:
# 첫 번째 대사 생성
dialogue = scene["dialogue"][0]

print("=" * 50)
print("음성 생성 파라미터")
print("=" * 50)
print(f"캐릭터: {char['name']}")
print(f"대사: {dialogue['text']}")
print(f"감정: {dialogue['emotion']}")
print(f"Voice ID: {char.get('voice_profile', 'default')}")

print("\n🔄 음성 생성 중...")

try:
    voice_path = tts_client.generate_speech(
        text=dialogue["text"],
        voice_id=char.get("voice_profile", "default"),
        emotion=dialogue["emotion"],
        output_filename=f"app/data/outputs/{run_id}/scene_1_line_1.mp3",
    )

    print(f"\n✅ 음성 생성 완료: {voice_path}")

    # 오디오 재생
    from IPython.display import Audio, display

    if Path(voice_path).exists():
        display(Audio(filename=voice_path))

except Exception as e:
    print(f"\n❌ 음성 생성 실패: {e}")

음성 생성 파라미터
캐릭터: 코스모냥이
대사: 나는 우주를 탐험할 준비가 되었어! 텔레포트 기계를 작동시켜 보자! 
감정: excited
Voice ID: default

🔄 음성 생성 중...

❌ 음성 생성 실패: name 'tts_client' is not defined


## 5. 통합 테스트 - 전체 씬 생성

모든 씬의 에셋을 생성하고 JSON을 업데이트합니다.

In [ ]:
# 첫 번째 씬의 이미지 생성 테스트
scene = layout["scenes"][0]
char = layout["characters"][0]

print("=" * 50)
print("이미지 생성 프롬프트")
print("=" * 50)

# 프롬프트 구성
art_style = layout["metadata"]["art_style"]
persona = char["persona"]
char_seed = char["seed"]
bg_seed = scene["bg_seed"]

image_prompt = f"{art_style}, {persona}, standing, neutral expression"

print(f"캐릭터: {char['name']}")
print(f"Persona: {persona}")
print(f"Character Seed: {char_seed}")
print(f"Background Seed: {bg_seed}")
print(f"\n최종 프롬프트: {image_prompt}")

print("\n🔄 이미지 생성 중...")

try:
    image_path = image_client.generate_image(
        prompt=image_prompt,
        seed=char_seed,
        output_prefix=f"app/data/outputs/{run_id}/scene_1_center",
    )

    print(f"\n✅ 이미지 생성 완료: {image_path}")

    # 이미지 표시 (Jupyter에서)
    from IPython.display import Image, display

    if Path(image_path).exists():
        display(Image(filename=image_path, width=300))

except Exception as e:
    print(f"\n❌ 이미지 생성 실패: {e}")
    print("\n💡 ComfyUI가 실행 중인지 확인하세요: http://localhost:8188")
    import traceback
    traceback.print_exc()

In [ ]:
# 업데이트된 JSON 저장
updated_json_path = Path(json_path).parent / "layout_with_assets.json"
with open(updated_json_path, "w", encoding="utf-8") as f:
    json.dump(layout, f, indent=2, ensure_ascii=False)

print(f"✅ 업데이트된 JSON 저장: {updated_json_path}")

# 최종 결과 요약
print("\n" + "=" * 50)
print("최종 결과 요약")
print("=" * 50)

total_images = sum(len(s["images"]) for s in layout["scenes"])
total_voices = sum(len(s["dialogue"]) for s in layout["scenes"])

print(f"생성된 이미지: {total_images}개")
print(f"생성된 음성: {total_voices}개")
print(f"배경음악: 1개")
print(f"\n모든 에셋 저장 위치: app/data/outputs/{run_id}/")

## 6. 프롬프트 분석 및 최적화

생성된 프롬프트를 분석하고 개선 방안을 제안합니다.

In [ ]:
# 프롬프트 분석 (스키마 v3)
print("=" * 50)
print("프롬프트 분석 (v3)")
print("=" * 50)

print("\n1. GPT 프롬프트 (2단계 생성)")
print("-" * 50)
print("1단계: 캐릭터 정의 생성 (characters.json)")
print("  - char_id, name, appearance, personality, voice_profile, seed")
print("  - appearance: 이미지 생성 기본 프롬프트")
print("\n2단계: 플롯 생성 (plot.csv)")
print("  - scene_id, char_id, expression, pose, text, text_type, emotion, subtitle_position, duration_ms")
print("  - expression/pose: 씬별 이미지 제어")

print("\n2. 이미지 생성 프롬프트 (v3: appearance + expression + pose)")
print("-" * 50)
for char in layout["characters"]:
    # CSV에서 해당 캐릭터의 첫 번째 씬 정보 가져오기
    df = pd.read_csv(csv_path)
    char_scenes = df[df['char_id'] == char['char_id']]
    
    if not char_scenes.empty:
        first_scene = char_scenes.iloc[0]
        expression = first_scene['expression']
        pose = first_scene['pose']
        
        if expression != "none" and pose != "none":
            prompt = f"{art_style}, {char['persona']}, {expression} expression, {pose} pose"
        else:
            prompt = f"{art_style}, {char['persona']}"
        
        print(f"[{char['name']}] {prompt}")
        print(f"  구성: appearance + expression + pose")
        print(f"  Seed: {char['seed']}")

print("\n3. 음성 생성 파라미터")
print("-" * 50)
for scene in layout["scenes"]:
    for dialogue in scene["dialogue"]:
        char = next(
            c for c in layout["characters"] if c["char_id"] == dialogue["char_id"]
        )
        print(f"[{char['name']}] {dialogue['text'][:30]}...")
        print(f"  감정: {dialogue['emotion']}")
        print(f"  길이: {len(dialogue['text'])} 문자")

print("\n4. 음악 생성 프롬프트")
print("-" * 50)
genre = layout["metadata"]["music_genre"]
duration_ms = layout["timeline"]["total_duration_ms"]
duration_sec = duration_ms / 1000

print(f"[ElevenLabs Sound Effects API 권장]")
print(f"프롬프트 예시: calm and peaceful {genre} atmospheric background music, instrumental, no vocals, looping")
print(f"길이: {duration_sec}초 (최대 22초)")
print(f"API 파라미터: duration_seconds, prompt_influence=0.3")

print("\n" + "=" * 50)
print("스키마 v3 최적화 포인트")
print("=" * 50)
print("1. 캐릭터 외형 재사용: characters.json → 모든 씬에서 일관성")
print("2. 씬별 세밀 제어: expression + pose → 같은 캐릭터, 다양한 연출")
print("3. 이미지 프롬프트 자동 조합: appearance + expression + pose")
print("4. 음악: ElevenLabs 권장 (TTS와 통합, 저렴)")
print("5. CSV 스키마: text_type으로 dialogue/narration 명확히 구분")

## 7. 정리 및 다음 단계

생성된 모든 파일과 다음 단계를 확인합니다.

In [ ]:
# 생성된 파일 목록
output_dir = Path(f"app/data/outputs/{run_id}")

print("=" * 50)
print(f"생성된 파일 목록 ({output_dir})")
print("=" * 50)

if output_dir.exists():
    files = sorted(output_dir.glob("*"))

    for file in files:
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"{file.name:40s} {size_mb:>8.2f} MB")

    print(f"\n총 {len(files)}개 파일")
else:
    print("⚠️  출력 디렉토리가 없습니다.")

print("\n" + "=" * 50)
print("다음 단계")
print("=" * 50)
print("1. Director Agent로 영상 합성 (MoviePy)")
print("2. QA Agent로 품질 검수")
print("3. 최종 영상 확인")
print("\n💡 전체 파이프라인 실행은 FastAPI + Celery로 진행하세요.")